In [ ]:
def create_sql_import(data_list):
    '''
    Write a SQL dump file that can be used to insert all the records from a given endpoint list into a database.
    Include a CREATE TABLE statement, and an INSERT statement for all the records in the given list.
    Prompt for name of table (and table.sql script) name.
    
    Input: list of dictionaries
    Output: SQL script file
    Return:  None
    '''
    
    tablename = input('Target table (it, and a SQL file by the same name, will be overwritten if exists): ') 
    sql_filename = tablename + '.sql'
    
    # This function assumes the order of key/value pairs in the dict remains stable.
    
    # Create start of CREATE TABLE command. 
    # Add unique identifier ID as the first column.
    create_table_string = ' '.join(['DROP TABLE IF EXISTS', tablename, ';\n\nCREATE TABLE', tablename, '(id INT NOT NULL'])
           
    # Get list of columns from first dictionary's keys, and assign a data type to each.  
    # Assume that all columns are of type INT except where '_date' or '_time' is 
    # in the column name, then set the data type accordingly.
    # ---> NOTE:  MS Access and MS SQL may not support the DATE and TIME datatypes. <---

    for key in data_list[0].keys():
        if '_date' in key:
            datatype = 'DATE'
        elif '_time' in key:
            datatype = 'TIME'
        else:
            datatype = 'INT'
            
        create_table_string += ', ' + key + ' ' + datatype

    create_table_string += ',\nCONSTRAINT PK_id PRIMARY KEY (id));\n\n'

    with open(sql_filename, 'w', newline='\n') as f:
        f.write(create_table_string)
        
    
    # Loop through list of dictionaries, writing to an INSERT statement in the SQL script.
    # Generate and add unique ID to each insert, since 'AUTO INCREMENT' is not standard on all RDBMSs.
    
    record_count = len(data_list)
    
    with open(sql_filename, 'a', newline='') as f:
        f.write('INSERT INTO ' + tablename + ' VALUES \n')
        
        # This code is very clumsy so re-write it someday.
        
        i = 1   # used as PRIMARY ID generator, and to avoid writing final comma after list of records 
        for dict in data_list:
            f.write('(')
            dict_len = len(dict)
            
            f.write(str(i))  # PRIMARY KEY ID
            
            for k, v in dict.items():
                # Add quotes around date and time values
                if '_date' in k or '_time' in k:
                    f.write(",'" + v + "'" )
                else:
                    f.write(',' + v )
                
            if i == record_count:
                f.write(')')
            else:
                f.write('),')
            i += 1
        
        # finishl-off the insert statement
        f.write('\n;')
    
create_sql_import(sleep_list)  